# Second year Project: Bomberbot Decision Making

In [5]:
import bson
import datetime

# Read in the necessary files for further use
files = ['teachers', 'users', 'classrooms', 'grades', 'schools', 'loginhistories', 'students']
data = []
for file in files:
    file = open('bschools/' + file + '.bson', 'rb')
    b = bson.decode_all(file.read())
    data.append(b)

In [6]:
def convertList(listVar):
    '''Given a list, get the most common value/string in the list. When list is empty, return None. '''
    if listVar == []:
        return '0'
    else: 
        return Counter(listVar).most_common(1)[0][0]
    
def convertCountries(country, name):
    ''' Given a country name and a file name, convert the country name to the overall synonym of that country found
        from the file name. When new/other countries are made, it can be changed in the name file. '''
    textfile = open(name, "r")
    lines = textfile.read().split('\n\n')
    newLines = [line.split('\n') for line in lines]
    newObj = []
    for listLine in newLines:
        l = []
        for line in listLine:
            if line != '':
                l.append(line.lower().strip())
        newObj.append(list(set(l)))
    newObj[0][0] = newObj[0][0].replace(u'\ufeff', '')
    for countr in newObj:
        if country.lower().strip() in countr:
            return countr[0]

In [8]:
import pandas as pd
from collections import Counter
import math

features = {}

# Go through the teachers
for i in range(len(data[0])):
    teacher = data[0][i];
    teacherID = data[0][i]['_id']
    userID = teacher['user']
    # Find the user and schoolID of the teacher
    for user in data[1]:
        if userID == user['_id']:
            schoolID = user['school']
            break
    # Go through the schools and find the country, referral and whether the person bought it or not        
    for school in data[4]:
        if schoolID == school['_id']:
            if 'referral' in school.keys():
                if school['referral'] != '':
                    referral = 1
                else:
                    referral = 0
            else:
                referral = 0
            if 'country' in school.keys():
                country = school['country']
                country = convertCountries(country, 'SynonymsCountries.txt')
                if 'paymentDate' in school.keys():
                    bought = 1
                else:
                    bought = 0
    # Set the value for being a teacher or parent 
    if 'isParent' in teacher.keys():
        if teacher['isParent']:
            teacherVal = 2
        else:
            teacherVal = 1
    else:
        teacherVal = 0
    # Set variables
    amountStudents = 0
    amountClassrooms = 0
    countGrades = 0
    avGrade = 0
    avTime = 0
    avTries = 0
    mobileT = []
    browserT = []
    osT = []
    mobileS = []
    browserS = []
    osS = []
    nloginsT = 0
    nloginsS = 0
    completeStudents = []
    studentIDS = []
    classrooms = data[2];
    for j in range(len(classrooms)):
        classroom = classrooms[j]
        if 'teacher' in classroom.keys():
            if teacherID == classroom['teacher']:
                students = classroom['students']
                completeStudents.extend(students)
                amountStudents += len(students)
                amountClassrooms += 1
    # Get specific userIDs of students
    for stud in data[6]:
        if stud['_id'] in completeStudents: 
            studentIDS.append(stud['user'])
    if len(completeStudents) > 0:
        # get student grades
        grades = data[3]
        dates = []
        times = []
        tries = []
        values = []
        for grade in grades:
            studentID = grade['student']
            if studentID in completeStudents:
                if 'date' in grade.keys():
                    dates.append(grade['date'])
                    times.append(grade['time'])
                    tries.append(grade['tries'])
                    values.append(grade['value'])
        # select grades in trial period
        if dates != []:
            trial_start = min(dates)
            thirty_days = datetime.timedelta(days=30)
            trial_end = trial_start + thirty_days
            # check if grade is in trial period
            for k in range(len(dates)):
                if dates[k] < trial_end:
                    if values[k] == 4:
                        val = 3
                    else:
                        val = values[k]
                    countGrades += 1
                    avGrade += val
                    avTime += times[k]
                    avTries += tries[k]
        else:
            trial_end = datetime.datetime.today()
    # Get login data
    for login in data[5]:
        if login['date'] < trial_end:
            if userID == login['user']:
                nloginsT += 1
                if login['browser']['mobile']:
                    mobileT.append(1)
                else:
                    mobileT.append(0)
                if 'version' in login['browser'].keys():
                    browserT.append(login['browser']['name'] + ' ' + login['browser']['version'])
                else:
                    browserT.append(login['browser']['name'])
                osT.append(login['os'])
            if login['user'] in studentIDS:
                nloginsS += 1
                if login['browser']['mobile']:
                    mobileS.append(1)
                else:
                    mobileS.append(0)
                if 'version' in login['browser'].keys():
                    browserS.append(login['browser']['name'] + ' ' + login['browser']['version'])
                else:
                    browserS.append(login['browser']['name'])
                osS.append(login['os'])
    # Set made variables in the correct way (normalizing, converting)
    if amountStudents > 0:
        averageStudents = amountStudents/amountClassrooms
    else:
        averageStudents = 0
    if countGrades > 0:
        avGrade = avGrade/countGrades 
        avTime = avTime/countGrades
        avTries = avTries/countGrades
    mobileT = convertList(mobileT)
    browserT = convertList(browserT)
    osT = convertList(osT)
    mobileS = convertList(mobileS)
    browserS = convertList(browserS)
    osS = convertList(osS)
    # Save the features of a row
    features[i] = (teacherVal, amountStudents, amountClassrooms, averageStudents, avGrade, avTime, avTries, referral, 
                   country, mobileT, browserT, osT, mobileS, browserS, osS, nloginsT, nloginsS, bought)

24078.939180999998


In [9]:
# Make the frame and show it 
dices = ['Teacher/Parent', 'Amount of Students', 'Amount Classrooms', 'Average Amount Students', 'Average Grade', 
         'Average Time', 'Average Tries', 'Referral', 'Country', 'Mobile Teacher', 'Browser Teacher', 'OS Teacher', 
         'Mobile Student', 'Browser Student', 'OS Student', 'Amount Logins Teacher', 'Amount Logins Students', 'Bought']
frame = pd.DataFrame(features, index=dices)
frame.T

,Teacher/Parent,Amount of Students,Amount Classrooms,Average Amount Students,Average Grade,Average Time,Average Tries,Referral,Country,Mobile Teacher,Browser Teacher,OS Teacher,Mobile Student,Browser Student,OS Student,Amount Logins Teacher,Amount Logins Students,Bought
0,1,12,3,4,0,8513,0,0,colobia,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0
5,0,2,1,2,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,nedrrl,0,0,0,0,0,0,0,0,0


In [10]:
# Save frame into pkl file for futher use
frame.to_pickle('frame.pkl')